This notebook has been executed with the following software, libraries and versions:

- Windows 10 (x86_64)
- Anaconda3-2023.09-0-Windows-x86_64
- Python 3.11.5 (64 bits)
- pandas v2.0.3
- scikit-learn v1.3.0
- matplotlib v3.7.2
- numpy v1.24.3

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 1. Reading and describing the dataset

In [2]:
mimic_iii_dataset_path = "../datasets/mimic-iii-dataset.csv"

In [3]:
df = pd.read_csv(mimic_iii_dataset_path)

In [4]:
df

,patient_id,patient_gender,patient_date_of_birth,patient_age,admission_id,admission_date,admission_date_of_the_previous_admission,readmission,exitus,admission_type,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
0,21,M,2047-04-04 00:00:00,87,111970,2135-01-30 20:50:00,2134-09-11 12:17:00,yes,yes,EMERGENCY,...,no,yes,no,no,yes,no,no,no,no,no
1,31,M,2036-05-17 00:00:00,72,128652,2108-08-22 23:27:00,NaN,no,yes,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
2,38,M,2090-08-31 00:00:00,75,185910,2166-08-10 00:28:00,NaN,no,no,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
3,38,M,2090-08-31 00:00:00,75,185910,2166-08-10 00:28:00,NaN,no,no,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
4,38,M,2090-08-31 00:00:00,75,185910,2166-08-10 00:28:00,NaN,no,no,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12791,99934,M,2092-03-18 00:00:00,17,176121,2110-02-28 03:47:00,NaN,no,no,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
12792,99935,M,2064-05-19 00:00:00,75,174624,2139-12-15 06:59:00,NaN,no,yes,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
12793,99946,M,2111-11-14 00:00:00,49,157197,2161-07-14 20:29:00,NaN,no,no,EMERGENCY,...,no,no,no,no,no,no,no,no,no,no
12794,99982,M,2091-10-02 00:00:00,65,183791,2157-02-16 17:31:00,2157-01-05 17:27:00,yes,no,EMERGENCY,...,no,no,no,no,no,no,no,yes,no,yes


In [5]:
df.dtypes

patient_id                                                           int64
patient_gender                                                      object
patient_date_of_birth                                               object
patient_age                                                          int64
admission_id                                                         int64
admission_date                                                      object
admission_date_of_the_previous_admission                            object
readmission                                                         object
exitus                                                              object
admission_type                                                      object
admission_location                                                  object
discharge_location                                                  object
admission_date_in_the_first_icu                                     object
time_between_admission_an

In [6]:
df.isnull().sum()

patient_id                                                            0
patient_gender                                                        0
patient_date_of_birth                                                 0
patient_age                                                           0
admission_id                                                          0
admission_date                                                        0
admission_date_of_the_previous_admission                           8580
readmission                                                           0
exitus                                                                0
admission_type                                                        0
admission_location                                                    0
discharge_location                                                    0
admission_date_in_the_first_icu                                     135
time_between_admission_and_first_icu                            

In [7]:
# Values of the columns.
for c in df.columns:
    if ('id' not in c.lower()) and ('date' not in c.lower()) and ('time' not in c.lower()):
        print("** COLUMN " + c.upper() + " **")
        print(df[c].value_counts(dropna=False))

** COLUMN PATIENT_GENDER **
patient_gender
M    7397
F    5399
Name: count, dtype: int64
** COLUMN PATIENT_AGE **
patient_age
90    548
68    345
69    312
76    304
64    303
     ... 
20     27
24     26
22     22
17     13
19     10
Name: count, Length: 74, dtype: int64
** COLUMN READMISSION **
readmission
no     8580
yes    4216
Name: count, dtype: int64
** COLUMN EXITUS **
exitus
no     10221
yes     2575
Name: count, dtype: int64
** COLUMN ADMISSION_TYPE **
admission_type
EMERGENCY    11081
ELECTIVE      1000
URGENT         509
NEWBORN        206
Name: count, dtype: int64
** COLUMN ADMISSION_LOCATION **
admission_location
EMERGENCY ROOM ADMIT         6160
TRANSFER FROM HOSP/EXTRAM    2769
CLINIC REFERRAL/PREMATURE    1945
PHYS REFERRAL/NORMAL DELI    1673
TRANSFER FROM SKILLED NUR     152
TRANSFER FROM OTHER HEALT      91
** INFO NOT AVAILABLE **        3
HMO REFERRAL/SICK               2
TRSF WITHIN THIS FACILITY       1
Name: count, dtype: int64
** COLUMN DISCHARGE_LOCATION **


# 2. Selection of attributes 

We delete the IDs and the dates.

In [8]:
df.drop(columns = ["patient_id", "patient_date_of_birth", "admission_id", "admission_date", "admission_date_of_the_previous_admission", "admission_date_in_the_first_icu", "culture_date", "culture_specimen_id", "culture_microorganism_id", "isolate_number"], inplace = True)

In [9]:
df

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,time_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
0,M,87,yes,yes,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,00:03:34,1,BLOOD CULTURE,...,no,yes,no,no,yes,no,no,no,no,no
1,M,72,no,yes,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,00:01:42,8,URINE,...,no,no,no,no,no,no,no,no,no,no
2,M,75,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,00:01:36,8,TISSUE,...,no,no,no,no,no,no,no,no,no,no
3,M,75,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,00:01:36,8,TISSUE,...,no,no,no,no,no,no,no,no,no,no
4,M,75,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,00:01:36,8,SPUTUM,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12791,M,17,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,00:00:56,2,ABSCESS,...,no,no,no,no,no,no,no,no,no,no
12792,M,75,no,yes,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,02:16:19,12,MRSA SCREEN,...,no,no,no,no,no,no,no,no,no,no
12793,M,49,no,no,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,00:01:56,7,URINE,...,no,no,no,no,no,no,no,no,no,no
12794,M,65,yes,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,SHORT TERM HOSPITAL,00:01:13,2,URINE,...,no,no,no,no,no,no,no,yes,no,yes


In [10]:
df.dtypes

patient_gender                                                     object
patient_age                                                         int64
readmission                                                        object
exitus                                                             object
admission_type                                                     object
admission_location                                                 object
discharge_location                                                 object
time_between_admission_and_first_icu                               object
culture_month                                                       int64
culture_specimen_type_description                                  object
culture_microorganism_name                                         object
culture_susceptibility                                             object
service_when_culture                                               object
icu_when_culture                      

In [11]:
df.isnull().sum()

patient_gender                                                       0
patient_age                                                          0
readmission                                                          0
exitus                                                               0
admission_type                                                       0
admission_location                                                   0
discharge_location                                                   0
time_between_admission_and_first_icu                               135
culture_month                                                        0
culture_specimen_type_description                                    0
culture_microorganism_name                                           0
culture_susceptibility                                               0
service_when_culture                                                 0
icu_when_culture                                                     0
treate

# 3. Cleaning and transforming data

## 3.1. Deletion of duplicated columns and rows

In [12]:
print("Duplicate columns: " + str(sum(df.columns.duplicated())))
print("Duplicate rows: " + str(sum(df.duplicated())))

Duplicate columns: 0
Duplicate rows: 2006


In [13]:
df.drop_duplicates(inplace=True)
# We reset/regenerate the indexes after dropping duplicates.
df.reset_index(drop=True,inplace=True)

In [14]:
print("Duplicate columns: " + str(sum(df.columns.duplicated())))
print("Duplicate rows: " + str(sum(df.duplicated())))

Duplicate columns: 0
Duplicate rows: 0


In [15]:
df

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,time_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
0,M,87,yes,yes,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,00:03:34,1,BLOOD CULTURE,...,no,yes,no,no,yes,no,no,no,no,no
1,M,72,no,yes,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,00:01:42,8,URINE,...,no,no,no,no,no,no,no,no,no,no
2,M,75,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,00:01:36,8,TISSUE,...,no,no,no,no,no,no,no,no,no,no
3,M,75,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,00:01:36,8,SPUTUM,...,no,no,no,no,no,no,no,no,no,no
4,M,56,no,no,ELECTIVE,PHYS REFERRAL/NORMAL DELI,DISC-TRAN TO FEDERAL HC,2 days 20:04:40,1,PLEURAL FLUID,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,M,71,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,1 day 15:01:20,5,Staph aureus Screen,...,no,no,no,no,no,no,no,no,no,no
10786,M,17,no,no,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,00:00:56,2,ABSCESS,...,no,no,no,no,no,no,no,no,no,no
10787,M,75,no,yes,EMERGENCY,EMERGENCY ROOM ADMIT,DEAD/EXPIRED,02:16:19,12,MRSA SCREEN,...,no,no,no,no,no,no,no,no,no,no
10788,M,49,no,no,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,00:01:56,7,URINE,...,no,no,no,no,no,no,no,no,no,no


In [16]:
df.dtypes

patient_gender                                                     object
patient_age                                                         int64
readmission                                                        object
exitus                                                             object
admission_type                                                     object
admission_location                                                 object
discharge_location                                                 object
time_between_admission_and_first_icu                               object
culture_month                                                       int64
culture_specimen_type_description                                  object
culture_microorganism_name                                         object
culture_susceptibility                                             object
service_when_culture                                               object
icu_when_culture                      

In [17]:
df.isnull().sum()

patient_gender                                                       0
patient_age                                                          0
readmission                                                          0
exitus                                                               0
admission_type                                                       0
admission_location                                                   0
discharge_location                                                   0
time_between_admission_and_first_icu                               120
culture_month                                                        0
culture_specimen_type_description                                    0
culture_microorganism_name                                           0
culture_susceptibility                                               0
service_when_culture                                                 0
icu_when_culture                                                     0
treate

## 3.2. Deletion of empty attributes or with only one value

In [18]:
# Values of the columns.
for c in df.columns:
    print("** COLUMN " + c.upper() + " **")
    print(df[c].value_counts(dropna=False))

** COLUMN PATIENT_GENDER **
patient_gender
M    6141
F    4649
Name: count, dtype: int64
** COLUMN PATIENT_AGE **
patient_age
90    499
68    301
69    272
76    258
72    255
     ... 
20     21
22     20
24     20
17     10
19      9
Name: count, Length: 74, dtype: int64
** COLUMN READMISSION **
readmission
no     7166
yes    3624
Name: count, dtype: int64
** COLUMN EXITUS **
exitus
no     8610
yes    2180
Name: count, dtype: int64
** COLUMN ADMISSION_TYPE **
admission_type
EMERGENCY    9389
ELECTIVE      835
URGENT        399
NEWBORN       167
Name: count, dtype: int64
** COLUMN ADMISSION_LOCATION **
admission_location
EMERGENCY ROOM ADMIT         5248
TRANSFER FROM HOSP/EXTRAM    2260
CLINIC REFERRAL/PREMATURE    1683
PHYS REFERRAL/NORMAL DELI    1381
TRANSFER FROM SKILLED NUR     130
TRANSFER FROM OTHER HEALT      83
** INFO NOT AVAILABLE **        3
HMO REFERRAL/SICK               1
TRSF WITHIN THIS FACILITY       1
Name: count, dtype: int64
** COLUMN DISCHARGE_LOCATION **
discha

There are not empty attributes or with only one value.

## 3.3. Treatment of missing values 

In [19]:
df.dtypes

patient_gender                                                     object
patient_age                                                         int64
readmission                                                        object
exitus                                                             object
admission_type                                                     object
admission_location                                                 object
discharge_location                                                 object
time_between_admission_and_first_icu                               object
culture_month                                                       int64
culture_specimen_type_description                                  object
culture_microorganism_name                                         object
culture_susceptibility                                             object
service_when_culture                                               object
icu_when_culture                      

In [20]:
df.isnull().sum()

patient_gender                                                       0
patient_age                                                          0
readmission                                                          0
exitus                                                               0
admission_type                                                       0
admission_location                                                   0
discharge_location                                                   0
time_between_admission_and_first_icu                               120
culture_month                                                        0
culture_specimen_type_description                                    0
culture_microorganism_name                                           0
culture_susceptibility                                               0
service_when_culture                                                 0
icu_when_culture                                                     0
treate

### Attribute 'time_between_admission_and_first_icu'

These rows correspond to patients admitted to the hospital, but with no information in the "icustays" table (because of any reason). However, the rows contain other data.

Therefore, we impute the missing values with -1.

In [21]:
df[df["time_between_admission_and_first_icu"].isnull()]

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,time_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
261,M,86,yes,no,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,NaN,1,SWAB,...,no,yes,no,no,yes,yes,yes,no,no,no
450,M,45,no,no,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,NaN,7,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
451,M,45,no,no,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,NaN,7,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
513,M,69,no,no,EMERGENCY,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,NaN,4,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
514,M,69,no,no,EMERGENCY,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,NaN,4,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10012,F,55,yes,no,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,NaN,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10013,F,55,yes,no,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,NaN,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10279,M,57,no,no,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,NaN,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no
10280,M,57,no,no,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,NaN,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no


In [22]:
df["time_between_admission_and_first_icu"].fillna("-1 days +23:00:00", inplace=True)

In [23]:
df[df["time_between_admission_and_first_icu"] == "-1 days +23:00:00"]

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,time_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
261,M,86,yes,no,URGENT,TRANSFER FROM HOSP/EXTRAM,SNF,-1 days +23:00:00,1,SWAB,...,no,yes,no,no,yes,yes,yes,no,no,no
450,M,45,no,no,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,-1 days +23:00:00,7,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
451,M,45,no,no,EMERGENCY,EMERGENCY ROOM ADMIT,HOME WITH HOME IV PROVIDR,-1 days +23:00:00,7,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
513,M,69,no,no,EMERGENCY,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,-1 days +23:00:00,4,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
514,M,69,no,no,EMERGENCY,PHYS REFERRAL/NORMAL DELI,REHAB/DISTINCT PART HOSP,-1 days +23:00:00,4,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10012,F,55,yes,no,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,-1 days +23:00:00,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10013,F,55,yes,no,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,-1 days +23:00:00,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10279,M,57,no,no,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,-1 days +23:00:00,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no
10280,M,57,no,no,EMERGENCY,CLINIC REFERRAL/PREMATURE,SNF,-1 days +23:00:00,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no


In [24]:
df.isnull().sum()

patient_gender                                                     0
patient_age                                                        0
readmission                                                        0
exitus                                                             0
admission_type                                                     0
admission_location                                                 0
discharge_location                                                 0
time_between_admission_and_first_icu                               0
culture_month                                                      0
culture_specimen_type_description                                  0
culture_microorganism_name                                         0
culture_susceptibility                                             0
service_when_culture                                               0
icu_when_culture                                                   0
treated_with_vancomycin_in_previou

## 3.4. Transformation of attribute (types and/or values)

In [25]:
df.dtypes

patient_gender                                                     object
patient_age                                                         int64
readmission                                                        object
exitus                                                             object
admission_type                                                     object
admission_location                                                 object
discharge_location                                                 object
time_between_admission_and_first_icu                               object
culture_month                                                       int64
culture_specimen_type_description                                  object
culture_microorganism_name                                         object
culture_susceptibility                                             object
service_when_culture                                               object
icu_when_culture                      

### Attribute 'patient_age'

This attribute will have 3 values: 'CHILD' [0, 16), 'ADULT' [16, 65) and 'ELDERLY' [65,90].

In [26]:
df['patient_age']

0        87
1        72
2        75
3        75
4        56
         ..
10785    71
10786    17
10787    75
10788    49
10789    65
Name: patient_age, Length: 10790, dtype: int64

In [27]:
df['patient_age'] = pd.Series(np.digitize(df['patient_age'], [16,65])).apply(str).replace({"0" : "CHILD", "1" : "ADULT", "2" : "ELDERLY"})

In [28]:
df['patient_age']

0        ELDERLY
1        ELDERLY
2        ELDERLY
3        ELDERLY
4          ADULT
          ...   
10785    ELDERLY
10786      ADULT
10787    ELDERLY
10788      ADULT
10789    ELDERLY
Name: patient_age, Length: 10790, dtype: object

### Attribute 'admission_type'

In [29]:
df["admission_type"].value_counts()

admission_type
EMERGENCY    9389
ELECTIVE      835
URGENT        399
NEWBORN       167
Name: count, dtype: int64

According the the MIMIC-III documentation (https://mimic.mit.edu/docs/iii/tables/admissions/), "Emergency/urgent indicate unplanned medical care, and are often collapsed into a single category in studies".

In [30]:
df["admission_type"].replace({"URGENT" : "EMERGENCY"}, inplace=True)

In [31]:
df["admission_type"].value_counts()

admission_type
EMERGENCY    9788
ELECTIVE      835
NEWBORN       167
Name: count, dtype: int64

### Attribute 'admission_location'

In [32]:
df["admission_location"].value_counts()

admission_location
EMERGENCY ROOM ADMIT         5248
TRANSFER FROM HOSP/EXTRAM    2260
CLINIC REFERRAL/PREMATURE    1683
PHYS REFERRAL/NORMAL DELI    1381
TRANSFER FROM SKILLED NUR     130
TRANSFER FROM OTHER HEALT      83
** INFO NOT AVAILABLE **        3
HMO REFERRAL/SICK               1
TRSF WITHIN THIS FACILITY       1
Name: count, dtype: int64

In [33]:
df["admission_location"].replace({"** INFO NOT AVAILABLE **" : "NO_INFO"}, inplace = True)
df["admission_location"] = df["admission_location"].apply(lambda x : x.replace(" ", "_"))

In [34]:
df["admission_location"].value_counts()

admission_location
EMERGENCY_ROOM_ADMIT         5248
TRANSFER_FROM_HOSP/EXTRAM    2260
CLINIC_REFERRAL/PREMATURE    1683
PHYS_REFERRAL/NORMAL_DELI    1381
TRANSFER_FROM_SKILLED_NUR     130
TRANSFER_FROM_OTHER_HEALT      83
NO_INFO                         3
HMO_REFERRAL/SICK               1
TRSF_WITHIN_THIS_FACILITY       1
Name: count, dtype: int64

### Attribute 'discharge_location'

In [35]:
df["discharge_location"].value_counts()

discharge_location
REHAB/DISTINCT PART HOSP     2930
DEAD/EXPIRED                 2180
SNF                          1773
HOME HEALTH CARE             1616
LONG TERM CARE HOSPITAL       941
HOME                          642
DISC-TRAN CANCER/CHLDRN H     239
SHORT TERM HOSPITAL           185
HOSPICE-HOME                  105
HOSPICE-MEDICAL FACILITY       50
LEFT AGAINST MEDICAL ADVI      45
HOME WITH HOME IV PROVIDR      32
DISCH-TRAN TO PSYCH HOSP       25
DISC-TRAN TO FEDERAL HC        11
ICF                            11
OTHER FACILITY                  5
Name: count, dtype: int64

In [36]:
df["discharge_location"] = df["discharge_location"].apply(lambda x : x.replace(" ", "_"))

In [37]:
df["discharge_location"].value_counts()

discharge_location
REHAB/DISTINCT_PART_HOSP     2930
DEAD/EXPIRED                 2180
SNF                          1773
HOME_HEALTH_CARE             1616
LONG_TERM_CARE_HOSPITAL       941
HOME                          642
DISC-TRAN_CANCER/CHLDRN_H     239
SHORT_TERM_HOSPITAL           185
HOSPICE-HOME                  105
HOSPICE-MEDICAL_FACILITY       50
LEFT_AGAINST_MEDICAL_ADVI      45
HOME_WITH_HOME_IV_PROVIDR      32
DISCH-TRAN_TO_PSYCH_HOSP       25
DISC-TRAN_TO_FEDERAL_HC        11
ICF                            11
OTHER_FACILITY                  5
Name: count, dtype: int64

### Attribute 'time_between_admission_and_first_icu'

In [38]:
df["time_between_admission_and_first_icu"]

0               00:03:34
1               00:01:42
2               00:01:36
3               00:01:36
4        2 days 20:04:40
              ...       
10785     1 day 15:01:20
10786           00:00:56
10787           02:16:19
10788           00:01:56
10789           00:01:13
Name: time_between_admission_and_first_icu, Length: 10790, dtype: object

In [39]:
df["time_between_admission_and_first_icu"] = pd.to_timedelta(df["time_between_admission_and_first_icu"])

In [40]:
df["time_between_admission_and_first_icu"]

0       0 days 00:03:34
1       0 days 00:01:42
2       0 days 00:01:36
3       0 days 00:01:36
4       2 days 20:04:40
              ...      
10785   1 days 15:01:20
10786   0 days 00:00:56
10787   0 days 02:16:19
10788   0 days 00:01:56
10789   0 days 00:01:13
Name: time_between_admission_and_first_icu, Length: 10790, dtype: timedelta64[ns]

In [41]:
df[df["time_between_admission_and_first_icu"] == pd.Timedelta(hours=-1)]

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,time_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
261,M,ELDERLY,yes,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,SNF,-1 days +23:00:00,1,SWAB,...,no,yes,no,no,yes,yes,yes,no,no,no
450,M,ADULT,no,no,EMERGENCY,EMERGENCY_ROOM_ADMIT,HOME_WITH_HOME_IV_PROVIDR,-1 days +23:00:00,7,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
451,M,ADULT,no,no,EMERGENCY,EMERGENCY_ROOM_ADMIT,HOME_WITH_HOME_IV_PROVIDR,-1 days +23:00:00,7,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
513,M,ELDERLY,no,no,EMERGENCY,PHYS_REFERRAL/NORMAL_DELI,REHAB/DISTINCT_PART_HOSP,-1 days +23:00:00,4,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
514,M,ELDERLY,no,no,EMERGENCY,PHYS_REFERRAL/NORMAL_DELI,REHAB/DISTINCT_PART_HOSP,-1 days +23:00:00,4,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10012,F,ADULT,yes,no,ELECTIVE,PHYS_REFERRAL/NORMAL_DELI,SNF,-1 days +23:00:00,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10013,F,ADULT,yes,no,ELECTIVE,PHYS_REFERRAL/NORMAL_DELI,SNF,-1 days +23:00:00,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10279,M,ADULT,no,no,EMERGENCY,CLINIC_REFERRAL/PREMATURE,SNF,-1 days +23:00:00,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no
10280,M,ADULT,no,no,EMERGENCY,CLINIC_REFERRAL/PREMATURE,SNF,-1 days +23:00:00,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no


We transform the attribute to have only the number of hours. The new attribute name will be: "hours_between_admission_and_first_icu".

In [42]:
df["time_between_admission_and_first_icu"] = (df["time_between_admission_and_first_icu"] / pd.Timedelta(hours=1)).astype(int)

In [43]:
df.rename(columns={"time_between_admission_and_first_icu" : "hours_between_admission_and_first_icu"}, inplace=True)

In [44]:
df["hours_between_admission_and_first_icu"]

0         0
1         0
2         0
3         0
4        68
         ..
10785    39
10786     0
10787     2
10788     0
10789     0
Name: hours_between_admission_and_first_icu, Length: 10790, dtype: int32

In [45]:
df[df["hours_between_admission_and_first_icu"] == -1]

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
261,M,ELDERLY,yes,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,SNF,-1,1,SWAB,...,no,yes,no,no,yes,yes,yes,no,no,no
450,M,ADULT,no,no,EMERGENCY,EMERGENCY_ROOM_ADMIT,HOME_WITH_HOME_IV_PROVIDR,-1,7,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
451,M,ADULT,no,no,EMERGENCY,EMERGENCY_ROOM_ADMIT,HOME_WITH_HOME_IV_PROVIDR,-1,7,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
513,M,ELDERLY,no,no,EMERGENCY,PHYS_REFERRAL/NORMAL_DELI,REHAB/DISTINCT_PART_HOSP,-1,4,BLOOD CULTURE,...,no,no,no,no,no,no,no,no,no,no
514,M,ELDERLY,no,no,EMERGENCY,PHYS_REFERRAL/NORMAL_DELI,REHAB/DISTINCT_PART_HOSP,-1,4,CATHETER TIP-IV,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10012,F,ADULT,yes,no,ELECTIVE,PHYS_REFERRAL/NORMAL_DELI,SNF,-1,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10013,F,ADULT,yes,no,ELECTIVE,PHYS_REFERRAL/NORMAL_DELI,SNF,-1,2,URINE,...,no,yes,yes,yes,no,yes,no,no,no,no
10279,M,ADULT,no,no,EMERGENCY,CLINIC_REFERRAL/PREMATURE,SNF,-1,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no
10280,M,ADULT,no,no,EMERGENCY,CLINIC_REFERRAL/PREMATURE,SNF,-1,11,TISSUE,...,no,no,no,no,no,no,no,no,no,no


We discretize the attribute.

In [46]:
df['hours_between_admission_and_first_icu'].describe()

count    10790.000000
mean        65.563670
std        183.452263
min         -1.000000
25%          0.000000
50%          0.000000
75%         43.000000
max       3269.000000
Name: hours_between_admission_and_first_icu, dtype: float64

In [47]:
df['hours_between_admission_and_first_icu'].value_counts()

hours_between_admission_and_first_icu
 0      5790
 2       187
 1       187
 3       141
-1       120
        ... 
 231       1
 683       1
 606       1
 598       1
 316       1
Name: count, Length: 518, dtype: int64

We see that the majority of the values are 0.

Therefore, we use the 33rd and 66th percentiles without considering the values equal to -1 and 0.

In [48]:
# Percentiles without considering the value -1.
att_33th_percentile = np.percentile([x for x in df['hours_between_admission_and_first_icu'].to_list() if x>0], 33) # 33th percentile not considering the value -1. 
#days_between_admission_and_first_ICU__33th_percentile__without_minus1 = round(days_between_admission_and_first_ICU__33th_percentile__without_minus1, 2)
att_66th_percentile = np.percentile([x for x in df['hours_between_admission_and_first_icu'].to_list() if x>0], 66) # 66th percentile not considering the value -1.
#days_between_admission_and_first_ICU__66th_percentile__without_minus1 = round(days_between_admission_and_first_ICU__66th_percentile__without_minus1, 2)
print("33rd percentile: " + str(att_33th_percentile))
print("66rd percentile: " + str(att_66th_percentile))

33rd percentile: 22.0
66rd percentile: 112.0


In [49]:
print("Number of values equal to -1: " + str(len([x for x in df['hours_between_admission_and_first_icu'] if x == -1])))
print("Number of values equal to 0: " + str(len([x for x in df['hours_between_admission_and_first_icu'] if x == 0])))
print("Number of values in (0, 22]: " + str(len([x for x in df['hours_between_admission_and_first_icu'] if (x > 0) and (x <= 22)])))
print("Number of values in (22, 112]: " + str(len([x for x in df['hours_between_admission_and_first_icu'] if (x > 22) and (x <= 112)])))
print("Number of values greater than 112: " + str(len([x for x in df['hours_between_admission_and_first_icu'] if x > 112])))

Number of values equal to -1: 120
Number of values equal to 0: 5790
Number of values in (0, 22]: 1613
Number of values in (22, 112]: 1611
Number of values greater than 112: 1656


In [50]:
df['hours_between_admission_and_first_icu'] = pd.Series(np.digitize(df['hours_between_admission_and_first_icu'], [-0.5, 0.0, 22, 112], right=True)).apply(str).replace({"0" : "-1", "1" : "0", "2" : "0<x<=22", "3" : "22<x<=112", "4" : "112<x"})

In [51]:
df['hours_between_admission_and_first_icu'].value_counts()

hours_between_admission_and_first_icu
0            5790
112<x        1656
0<x<=22      1613
22<x<=112    1611
-1            120
Name: count, dtype: int64

### Attribute 'stay_in_icu_in_previous_admissions'

In [52]:
df["stay_in_icu_in_previous_admissions"]

0         5 days 21:38:28
1                00:00:00
2                00:00:00
3                00:00:00
4                00:00:00
               ...       
10785            00:00:00
10786            00:00:00
10787            00:00:00
10788            00:00:00
10789    44 days 02:47:27
Name: stay_in_icu_in_previous_admissions, Length: 10790, dtype: object

In [53]:
df["stay_in_icu_in_previous_admissions"] = pd.to_timedelta(df["stay_in_icu_in_previous_admissions"])

In [54]:
df["stay_in_icu_in_previous_admissions"]

0        5 days 21:38:28
1        0 days 00:00:00
2        0 days 00:00:00
3        0 days 00:00:00
4        0 days 00:00:00
              ...       
10785    0 days 00:00:00
10786    0 days 00:00:00
10787    0 days 00:00:00
10788    0 days 00:00:00
10789   44 days 02:47:27
Name: stay_in_icu_in_previous_admissions, Length: 10790, dtype: timedelta64[ns]

In [55]:
df[df["stay_in_icu_in_previous_admissions"] < pd.Timedelta(hours=0)]

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions


We transform the attribute to have only the number of hours. The new attribute name will be: "hours_of_stay_in_icu_in_previous_admissions".

In [56]:
df["stay_in_icu_in_previous_admissions"] = (df["stay_in_icu_in_previous_admissions"] / pd.Timedelta(hours=1)).astype(int)

In [57]:
df.rename(columns={"stay_in_icu_in_previous_admissions" : "hours_of_stay_in_icu_in_previous_admission"}, inplace=True)

In [58]:
df["hours_of_stay_in_icu_in_previous_admission"]

0         141
1           0
2           0
3           0
4           0
         ... 
10785       0
10786       0
10787       0
10788       0
10789    1058
Name: hours_of_stay_in_icu_in_previous_admission, Length: 10790, dtype: int32

In [59]:
df[df["hours_of_stay_in_icu_in_previous_admission"] < 0]

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions


We discretize the attribute.

In [60]:
df['hours_of_stay_in_icu_in_previous_admission'].describe()

count    10790.000000
mean      2286.988323
std       8615.633467
min          0.000000
25%          0.000000
50%          0.000000
75%         67.750000
max      98436.000000
Name: hours_of_stay_in_icu_in_previous_admission, dtype: float64

In [61]:
df['hours_of_stay_in_icu_in_previous_admission'].value_counts()

hours_of_stay_in_icu_in_previous_admission
0        7278
25         52
23         37
24         36
29         36
         ... 
34072       1
15636       1
1124        1
58939       1
1058        1
Name: count, Length: 1322, dtype: int64

In [62]:
df['hours_of_stay_in_icu_in_previous_admission'].value_counts().sort_index()

hours_of_stay_in_icu_in_previous_admission
0        7278
1           2
2           1
3           1
4           1
         ... 
81391       3
83650       1
84400       1
93175       1
98436       1
Name: count, Length: 1322, dtype: int64

We use the 33rd and 66th percentiles without considering the values equal 0.

In [63]:
# Percentiles without considering the value 0.
att_33th_percentile = np.percentile([x for x in df['hours_of_stay_in_icu_in_previous_admission'].to_list() if x>0], 33) # 33th percentile not considering the value 0. 
att_66th_percentile = np.percentile([x for x in df['hours_of_stay_in_icu_in_previous_admission'].to_list() if x>0], 66) # 66th percentile not considering the value 0.
print("33rd percentile: " + str(att_33th_percentile))
print("66rd percentile: " + str(att_66th_percentile))

33rd percentile: 124.0
66rd percentile: 2023.8600000000024


In [64]:
print("Number of values equal to 0: " + str(len([x for x in df['hours_of_stay_in_icu_in_previous_admission'] if x == 0])))
print("Number of values in (0, 124]: " + str(len([x for x in df['hours_of_stay_in_icu_in_previous_admission'] if (x > 0) and (x <= 124)])))
print("Number of values in (124, 2024]: " + str(len([x for x in df['hours_of_stay_in_icu_in_previous_admission'] if (x > 124) and (x <= 2024)])))
print("Number of values greater than 2024: " + str(len([x for x in df['hours_of_stay_in_icu_in_previous_admission'] if x > 2024])))

Number of values equal to 0: 7278
Number of values in (0, 124]: 1164
Number of values in (124, 2024]: 1154
Number of values greater than 2024: 1194


In [65]:
df['hours_of_stay_in_icu_in_previous_admission'] = pd.Series(np.digitize(df['hours_of_stay_in_icu_in_previous_admission'], [0.5, 124, 2024], right=True)).apply(str).replace({"0" : "0", "1" : "0<x<=124", "2" : "124<x<=2024", "3" : "2024<x"})

In [66]:
df['hours_of_stay_in_icu_in_previous_admission'].value_counts()

hours_of_stay_in_icu_in_previous_admission
0              7278
2024<x         1194
0<x<=124       1164
124<x<=2024    1154
Name: count, dtype: int64

### Attribute 'culture_month'

In [67]:
df["culture_month"].value_counts()

culture_month
8     976
1     935
10    920
3     919
2     912
5     908
9     905
11    888
12    877
4     857
6     848
7     845
Name: count, dtype: int64

In [68]:
df['culture_month'] = df['culture_month'].apply(str).replace({"1" : "JANUARY" , \
                                                            "2" : "FEBRUARY" , \
                                                            "3" : "MARCH" , \
                                                            "4" : "APRIL" , \
                                                            "5" : "MAY" , \
                                                            "6" : "JUNE" , \
                                                            "7" : "JULY" , \
                                                            "8" : "AUGUST" , \
                                                            "9" : "SEPTEMBER" , \
                                                            "10" : "OCTOBER" , \
                                                            "11" : "NOVEMBER" , \
                                                            "12" : "DECEMBER"})

In [69]:
df["culture_month"].value_counts()

culture_month
AUGUST       976
JANUARY      935
OCTOBER      920
MARCH        919
FEBRUARY     912
MAY          908
SEPTEMBER    905
NOVEMBER     888
DECEMBER     877
APRIL        857
JUNE         848
JULY         845
Name: count, dtype: int64

### Attribute 'culture_specimen_type_description'

In [70]:
df["culture_specimen_type_description"].value_counts()

culture_specimen_type_description
BLOOD CULTURE                              2321
URINE                                      2067
SPUTUM                                     1823
SWAB                                       1718
CATHETER TIP-IV                             822
TISSUE                                      353
ABSCESS                                     188
PERITONEAL FLUID                            187
BRONCHOALVEOLAR LAVAGE                      180
MRSA SCREEN                                 169
BILE                                        166
FLUID,OTHER                                 161
BLOOD CULTURE - NEONATE                     125
PLEURAL FLUID                               100
BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)         65
CSF;SPINAL FLUID                             57
FLUID RECEIVED IN BLOOD CULTURE BOTTLES      46
BRONCHIAL WASHINGS                           42
FOREIGN BODY                                 32
FOOT CULTURE                                 26
JOINT 

In [71]:
df["culture_specimen_type_description"].replace({"BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)" : "BLOOD CULTURE (MYCO/F LYTIC BOTTLE)", "BLOOD CULTURE - NEONATE" : "BLOOD_CULTURE__NEONATE"}, inplace=True)
df["culture_specimen_type_description"] = df["culture_specimen_type_description"].apply(lambda x : x.replace(" ", "_"))

In [72]:
df["culture_specimen_type_description"].value_counts()

culture_specimen_type_description
BLOOD_CULTURE                              2321
URINE                                      2067
SPUTUM                                     1823
SWAB                                       1718
CATHETER_TIP-IV                             822
TISSUE                                      353
ABSCESS                                     188
PERITONEAL_FLUID                            187
BRONCHOALVEOLAR_LAVAGE                      180
MRSA_SCREEN                                 169
BILE                                        166
FLUID,OTHER                                 161
BLOOD_CULTURE__NEONATE                      125
PLEURAL_FLUID                               100
BLOOD_CULTURE_(MYCO/F_LYTIC_BOTTLE)          65
CSF;SPINAL_FLUID                             57
FLUID_RECEIVED_IN_BLOOD_CULTURE_BOTTLES      46
BRONCHIAL_WASHINGS                           42
FOREIGN_BODY                                 32
FOOT_CULTURE                                 26
JOINT_

### Attribute 'culture_susceptibility'

When the susceptibily is 'I' (Intermediate), the clinical expert considers that que susceptibility is 'R' (Resistent) in order to be able to suministrate to the patient an antibiotic for resistent microorganisms (because the antibiotic for sensible microorganisms could not work).

In [73]:
df["culture_susceptibility"].value_counts()

culture_susceptibility
S    8901
R    1846
I      43
Name: count, dtype: int64

In [74]:
df["culture_susceptibility"].replace({"I" : "R"}, inplace=True)

In [75]:
df["culture_susceptibility"].value_counts()

culture_susceptibility
S    8901
R    1889
Name: count, dtype: int64

### Result

In [76]:
df

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
0,M,ELDERLY,yes,yes,EMERGENCY,EMERGENCY_ROOM_ADMIT,DEAD/EXPIRED,0,JANUARY,BLOOD_CULTURE,...,no,yes,no,no,yes,no,no,no,no,no
1,M,ELDERLY,no,yes,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,DEAD/EXPIRED,0,AUGUST,URINE,...,no,no,no,no,no,no,no,no,no,no
2,M,ELDERLY,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,LONG_TERM_CARE_HOSPITAL,0,AUGUST,TISSUE,...,no,no,no,no,no,no,no,no,no,no
3,M,ELDERLY,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,LONG_TERM_CARE_HOSPITAL,0,AUGUST,SPUTUM,...,no,no,no,no,no,no,no,no,no,no
4,M,ADULT,no,no,ELECTIVE,PHYS_REFERRAL/NORMAL_DELI,DISC-TRAN_TO_FEDERAL_HC,22<x<=112,JANUARY,PLEURAL_FLUID,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,M,ELDERLY,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,HOME_HEALTH_CARE,22<x<=112,MAY,Staph_aureus_Screen,...,no,no,no,no,no,no,no,no,no,no
10786,M,ADULT,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,HOME_HEALTH_CARE,0,FEBRUARY,ABSCESS,...,no,no,no,no,no,no,no,no,no,no
10787,M,ELDERLY,no,yes,EMERGENCY,EMERGENCY_ROOM_ADMIT,DEAD/EXPIRED,0<x<=22,DECEMBER,MRSA_SCREEN,...,no,no,no,no,no,no,no,no,no,no
10788,M,ADULT,no,no,EMERGENCY,CLINIC_REFERRAL/PREMATURE,SNF,0,JULY,URINE,...,no,no,no,no,no,no,no,no,no,no


In [77]:
df.dtypes

patient_gender                                                     object
patient_age                                                        object
readmission                                                        object
exitus                                                             object
admission_type                                                     object
admission_location                                                 object
discharge_location                                                 object
hours_between_admission_and_first_icu                              object
culture_month                                                      object
culture_specimen_type_description                                  object
culture_microorganism_name                                         object
culture_susceptibility                                             object
service_when_culture                                               object
icu_when_culture                      

In [78]:
df.isna().sum()

patient_gender                                                     0
patient_age                                                        0
readmission                                                        0
exitus                                                             0
admission_type                                                     0
admission_location                                                 0
discharge_location                                                 0
hours_between_admission_and_first_icu                              0
culture_month                                                      0
culture_specimen_type_description                                  0
culture_microorganism_name                                         0
culture_susceptibility                                             0
service_when_culture                                               0
icu_when_culture                                                   0
treated_with_vancomycin_in_previou

# 4. Correlation between attributes

In [79]:
numeric_df = df.copy()

In [80]:
le = LabelEncoder()
for c in numeric_df.columns:
    numeric_df[c] = le.fit_transform(numeric_df[c])

In [81]:
numeric_df

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
0,1,2,1,1,1,1,0,1,4,4,...,0,1,0,0,1,0,0,0,0,0
1,1,2,0,1,1,5,0,1,1,38,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,0,1,5,11,1,1,36,...,0,0,0,0,0,0,0,0,0,0
3,1,2,0,0,1,5,11,1,1,29,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,4,2,4,4,28,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,1,2,0,0,1,5,5,4,8,33,...,0,0,0,0,0,0,0,0,0,0
10786,1,0,0,0,1,5,5,1,3,0,...,0,0,0,0,0,0,0,0,0,0
10787,1,2,0,1,1,1,0,2,2,24,...,0,0,0,0,0,0,0,0,0,0
10788,1,0,0,0,1,0,15,1,5,38,...,0,0,0,0,0,0,0,0,0,0


In [82]:
corr_matrix = numeric_df.corr().abs().applymap(lambda x : 'no' if x<0.8 else 'YES')
corr_matrix

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,previous_exposure_to_glycopeptides,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
patient_gender,YES,no,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
patient_age,no,YES,no,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
readmission,no,no,YES,no,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
exitus,no,no,no,YES,no,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
admission_type,no,no,no,no,YES,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
admission_location,no,no,no,no,no,YES,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
discharge_location,no,no,no,no,no,no,YES,no,no,no,...,no,no,no,no,no,no,no,no,no,no
hours_between_admission_and_first_icu,no,no,no,no,no,no,no,YES,no,no,...,no,no,no,no,no,no,no,no,no,no
culture_month,no,no,no,no,no,no,no,no,YES,no,...,no,no,no,no,no,no,no,no,no,no
culture_specimen_type_description,no,no,no,no,no,no,no,no,no,YES,...,no,no,no,no,no,no,no,no,no,no


In [83]:
for i in range(len(df.columns)-1):
    for j in range(i+1,len(df.columns)):
        current_cell = corr_matrix.iloc[i][j]
        if current_cell == "YES":
            print("Attributes '" + df.columns[i] + "' and '" + df.columns[j] + "' are correlated.")

Attributes 'readmission' and 'hours_of_stay_in_icu_in_previous_admission' are correlated.
Attributes 'treated_with_vancomycin_in_previous_admissions' and 'previous_exposure_to_glycopeptides' are correlated.


In [84]:
# We then delete the following attributes:
# - "previous_exposure_to_glycopeptides" attribute since it is more general and this research is focused on the vancomycin.
# - "hours_of_stay_in_icu_in_previous_admission" because it has more values than "readmission".
print("Number of attributes: " + str(len(df.columns)))
df.drop(columns=["previous_exposure_to_glycopeptides", "hours_of_stay_in_icu_in_previous_admission"], inplace=True)
print("Number of attributes: " + str(len(df.columns)))

Number of attributes: 34
Number of attributes: 32


# 5. Summarization and description of data

In [85]:
df

,patient_gender,patient_age,readmission,exitus,admission_type,admission_location,discharge_location,hours_between_admission_and_first_icu,culture_month,culture_specimen_type_description,...,previous_exposure_to_third_generation_cephalosporins,previous_exposure_to_carbapenems,previous_exposure_to_fluorquinolones,previous_exposure_to_aminoglycosides,previous_exposure_to_b_lactam,previous_exposure_to_antifungal_agents,mechanical_ventilation_in_previous_admissions,organ_transplant_in_previous_admissions,hematopoietic_transplant_in_previous_admissions,catheter_in_previous_admissions
0,M,ELDERLY,yes,yes,EMERGENCY,EMERGENCY_ROOM_ADMIT,DEAD/EXPIRED,0,JANUARY,BLOOD_CULTURE,...,no,no,yes,no,no,yes,no,no,no,no
1,M,ELDERLY,no,yes,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,DEAD/EXPIRED,0,AUGUST,URINE,...,no,no,no,no,no,no,no,no,no,no
2,M,ELDERLY,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,LONG_TERM_CARE_HOSPITAL,0,AUGUST,TISSUE,...,no,no,no,no,no,no,no,no,no,no
3,M,ELDERLY,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,LONG_TERM_CARE_HOSPITAL,0,AUGUST,SPUTUM,...,no,no,no,no,no,no,no,no,no,no
4,M,ADULT,no,no,ELECTIVE,PHYS_REFERRAL/NORMAL_DELI,DISC-TRAN_TO_FEDERAL_HC,22<x<=112,JANUARY,PLEURAL_FLUID,...,no,no,no,no,no,no,no,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10785,M,ELDERLY,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,HOME_HEALTH_CARE,22<x<=112,MAY,Staph_aureus_Screen,...,no,no,no,no,no,no,no,no,no,no
10786,M,ADULT,no,no,EMERGENCY,TRANSFER_FROM_HOSP/EXTRAM,HOME_HEALTH_CARE,0,FEBRUARY,ABSCESS,...,no,no,no,no,no,no,no,no,no,no
10787,M,ELDERLY,no,yes,EMERGENCY,EMERGENCY_ROOM_ADMIT,DEAD/EXPIRED,0<x<=22,DECEMBER,MRSA_SCREEN,...,no,no,no,no,no,no,no,no,no,no
10788,M,ADULT,no,no,EMERGENCY,CLINIC_REFERRAL/PREMATURE,SNF,0,JULY,URINE,...,no,no,no,no,no,no,no,no,no,no


In [86]:
df.dtypes

patient_gender                                                     object
patient_age                                                        object
readmission                                                        object
exitus                                                             object
admission_type                                                     object
admission_location                                                 object
discharge_location                                                 object
hours_between_admission_and_first_icu                              object
culture_month                                                      object
culture_specimen_type_description                                  object
culture_microorganism_name                                         object
culture_susceptibility                                             object
service_when_culture                                               object
icu_when_culture                      

In [87]:
print("Number of columns: " + str(len(df.columns)))
print("Number of rows: " + str(len(df)))

Number of columns: 32
Number of rows: 10790


In [88]:
# Values of the columns.
for c in df.columns:
    print("** COLUMN " + c.upper() + " **")
    print(df[c].value_counts(dropna=False))

** COLUMN PATIENT_GENDER **
patient_gender
M    6141
F    4649
Name: count, dtype: int64
** COLUMN PATIENT_AGE **
patient_age
ELDERLY    5558
ADULT      5059
CHILD       173
Name: count, dtype: int64
** COLUMN READMISSION **
readmission
no     7166
yes    3624
Name: count, dtype: int64
** COLUMN EXITUS **
exitus
no     8610
yes    2180
Name: count, dtype: int64
** COLUMN ADMISSION_TYPE **
admission_type
EMERGENCY    9788
ELECTIVE      835
NEWBORN       167
Name: count, dtype: int64
** COLUMN ADMISSION_LOCATION **
admission_location
EMERGENCY_ROOM_ADMIT         5248
TRANSFER_FROM_HOSP/EXTRAM    2260
CLINIC_REFERRAL/PREMATURE    1683
PHYS_REFERRAL/NORMAL_DELI    1381
TRANSFER_FROM_SKILLED_NUR     130
TRANSFER_FROM_OTHER_HEALT      83
NO_INFO                         3
HMO_REFERRAL/SICK               1
TRSF_WITHIN_THIS_FACILITY       1
Name: count, dtype: int64
** COLUMN DISCHARGE_LOCATION **
discharge_location
REHAB/DISTINCT_PART_HOSP     2930
DEAD/EXPIRED                 2180
SNF        

# 6. Write the mining view

In [89]:
mimic_iii_mining_view_path = "../datasets/mimic-iii-mining-view.csv"

In [90]:
df.to_csv(mimic_iii_mining_view_path, index = False)